In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from sklearn.model_selection import train_test_split,GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn import linear_model
from sklearn import ensemble


In [2]:
df_all = pd.read_csv('cleaned_solar_tract.csv', encoding='latin-1')

In [3]:
df = df_all.dropna()

In [6]:
target = 'solar_system_count_residential'

misc = ['Unnamed: 0','fips', 'county', 'state']
solar = ['total_panel_area',  'total_panel_area_residential', 'total_panel_area_nonresidential',  
                   'solar_panel_area_divided_by_area', 'solar_panel_area_per_capita',
                'tile_count','tile_count_residential', 'tile_count_nonresidential','number_of_solar_system_per_household', 
                   'solar_system_count','solar_system_count_nonresidential']
non_residential = ['electricity_price_transportation','incentive_count_nonresidential',
                   'electricity_consume_commercial','electricity_price_overall', 'electricity_consume_industrial', 
                   'electricity_consume_total','electricity_price_industrial','electricity_price_commercial']

df = df.loc[:, ~(df.columns).isin(misc + solar + non_residential)]

In [7]:
def split_scale_data(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y)
    
    sclr_X = StandardScaler()
    sclr_X.fit(X_train[X_train.columns])
    
    sclr_y = StandardScaler()
    sclr_y.fit(y_train)

    X_train_scaled = pd.DataFrame(sclr_X.transform(X_train[X_train.columns]), columns = X_train.columns)
    X_test_scaled = pd.DataFrame(sclr_X.transform(X_test[X_test.columns]), columns = X_test.columns)

    y_train_scaled = sclr_y.transform(y_train)
    y_test_scaled = sclr_y.transform(y_test)
    
    return X_train_scaled, X_test_scaled, y_train_scaled, y_test_scaled, X_train, X_test, y_train, y_test, sclr_X, sclr_y

In [10]:
X = df.loc[:, ~(df.columns).isin([target])]
y = df[target]

X_train_scaled, X_test_scaled, y_train_scaled, y_test_scaled, X_train, X_test, y_train, y_test, sclr_X, sclr_y = split_scale_data(X, y.reshape(-1, 1))

c:\users\jordan landers\appdata\local\programs\python\python36-32\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  after removing the cwd from sys.path.


In [ ]:
pca = PCA(n_components=100)
pca.fit(X_train_scaled)

params = {'loss': ['ls', 'huber'], 
         'learning_rate': [.1] ,
         'criterion': ['friedman_mse', 'mae'],
         'max_depth': [6],
         'n_estimators':[400]
         }

import time
start = time.time()
gbr = ensemble.GradientBoostingRegressor(**params)
clf = GridSearchCV(gbr, params, cv = 3, n_jobs= -1, verbose = 10)
clf.fit(pca.transform(X_train_scaled), y_train_scaled)
clf.score(pca.transform(X_test_scaled), y_test_scaled)
end = time.time()
total_time = end-start
print('total time:', total_time)

Fitting 3 folds for each of 4 candidates, totalling 12 fits


[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed: 17.4min
[Parallel(n_jobs=-1)]: Done   3 out of  12 | elapsed: 17.6min remaining: 52.9min
[Parallel(n_jobs=-1)]: Done   5 out of  12 | elapsed: 17.9min remaining: 25.0min
